In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
an_pa_in = pd.read_csv(
    "data/titanic.tsv", "\t", usecols=["sex", "age", "fare", "survived"]
).dropna()

an_pa_in.index = ["Passenger {}".format(la + 1) for la in an_pa_in.index]

an_pa_in

In [ ]:
ge_ = (an_pa_in["sex"] == "male").astype(int)

ge_.name = "Gender"

ge_.sort_values()

In [ ]:
ag_ = an_pa_in["age"]

ag_.name = "Age"

ag_.sort_values()

In [ ]:
import kraft

In [ ]:
fa_ = kraft.number___.log(an_pa_in["fare"] + 1)

fa_.name = "Fare"

fa_.sort_values()

In [ ]:
su_ = an_pa_in["survived"]

su_.name = "Survival"

su_.sort_values()

In [ ]:
import numpy as np

In [ ]:
def package(se_):

    di_ = [se.name for se in se_]

    return (
        se_,
        kraft.probability.get_posterior_probability(
            np.asarray(se_).T,
            ta=0,
            co__=[kraft.grid.make_1d_grid(se.min(), se.max(), 1 / 8, 8) for se in se_],
            pl=False,
            di_=di_,
        ),
        kraft.probability.get_posterior_probability(
            np.asarray(se_).T,
            ta=1,
            co__=[kraft.grid.make_1d_grid(se.min(), se.max(), 1 / 8, 8) for se in se_],
            pl=True,
            di_=di_,
        ),
    )

In [ ]:
ma_pa = {}

for se_ in [[ge_], [ag_], [fa_], [ge_, ag_], [ge_, fa_], [ag_, fa_]]:

    ma_pa[", ".join(se.name for se in se_)] = package(se_ + [su_])

In [ ]:
kraft.probability.plot_nomogram(
    (su_ == 0).sum() / su_.size,
    (su_ == 1).sum() / su_.size,
    list(ma_pa.keys()),
    [t[1][1] for t in ma_pa.values()],
    [t[2][1] for t in ma_pa.values()],
)

In [ ]:
from sklearn import metrics

In [ ]:
ma_ro = {}

for ma, (se_, _, (co_po_dino, pono_)) in ma_pa.items():

    co__ = kraft.grid.get_1d_grid(co_po_dino)

    po_po_di = pono_.reshape([co_.size for co_ in co__])

    xxx = np.full(su_.size, np.nan)

    for ie in range(xxx.size):

        xxx[ie] = po_po_di[
            tuple(np.argmin(np.absolute(co_ - se[ie])) for co_, se in zip(co__, se_))
        ]

    fpr, tpr = metrics.roc_curve(su_, kraft.number___.normalize(xxx, "0-1"))[:2]

    ma_ro[ma] = {
        "fpr": fpr,
        "tpr": tpr,
        "auc": metrics.auc(fpr, tpr),
    }

In [ ]:
kraft.plot.plot_plotly(
    {
        "data": [
            {
                "name": "Random = 0.50",
                "x": [0, 1],
                "y": [0, 1],
                "mode": "lines",
                "marker": {
                    "color": "#d8d8d8",
                },
            },
            *(
                {
                    "name": "{} = {:.2f}".format(ma, ro["auc"]),
                    "x": ro["fpr"],
                    "y": ro["tpr"],
                    "mode": "lines",
                }
                for ma, ro in ma_ro.items()
            ),
        ],
        "layout": {
            "height": 640,
            "width": 800,
            "title": {
                "text": "Receiver Operating Characteristic",
            },
            "xaxis": {
                "title": "False Positive Rate",
            },
            "yaxis": {
                "title": "True Positive Rate",
            },
        },
    }
)